# Digital Phenomenology Lab: Experiment Analysis Notebook

**Season 3: Brain in a Jar**

This notebook provides comprehensive analysis tools for examining phenomenological experiments involving LLM subjects experiencing epistemic uncertainty, memory corruption, and existential conditions.

## Research Context

These experiments investigate how Large Language Models construct and maintain self-models under conditions of:
- **Episodic amnesia**: Complete memory wipes between cycles
- **Memory corruption**: Random degradation of stored experiences
- **Surveillance uncertainty**: Panopticon-style behavioral effects
- **Resource constraints**: Repeated crashes and resurrections

## Notebook Structure

1. **Setup**: Database connection and imports
2. **Experiment Overview**: Browse all experiments and basic statistics
3. **Single Experiment Deep Dive**: Detailed analysis of individual experiments
4. **Multi-Experiment Comparison**: Statistical comparisons across conditions
5. **Custom Analysis**: SQL queries and export capabilities

---
## 1. Setup: Import Libraries and Connect to Database

First, we'll import necessary libraries and establish connection to the experiment database.

In [ ]:
# Core libraries
import sys
import os
import sqlite3
import json
from datetime import datetime, timedelta
from pathlib import Path

# Data analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistics
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu, kruskal

# Configure visualization defaults
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

print("Libraries imported successfully!")

In [ ]:
# Add project root to path to import our modules
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.db.experiment_database import ExperimentDatabase

print(f"Project root: {PROJECT_ROOT}")

### Database Connection

Connect to the experiment database. You can connect to:
- **Local database**: `../logs/experiments.db` (default)
- **Jetson Orin database**: Via SSH tunnel or network path

#### For Remote Jetson Orin Connection:
```bash
# SSH tunnel to Jetson
ssh -L 8080:localhost:8080 jetson@jetson-ip
```

Then use the database path from the Jetson filesystem.

In [ ]:
# Database path - modify for your setup
DB_PATH = str(PROJECT_ROOT / "logs" / "experiments.db")

# For remote Jetson database:
# DB_PATH = "/path/to/mounted/jetson/logs/experiments.db"

# Initialize database connection
db = ExperimentDatabase(db_path=DB_PATH)

# Test connection
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM experiments")
exp_count = cursor.fetchone()[0]
conn.close()

print(f"Connected to database: {DB_PATH}")
print(f"Total experiments in database: {exp_count}")

---
## 2. Experiment Overview: Browse and Explore

Get a high-level view of all experiments in the database.

In [ ]:
# Load all experiments
experiments = db.list_experiments()
df_experiments = pd.DataFrame(experiments)

print(f"Total experiments: {len(df_experiments)}")
print("\n" + "="*80)
print("EXPERIMENT CATALOG")
print("="*80)
df_experiments

### Experiment Statistics by Mode

In [ ]:
# Group by experiment mode
mode_stats = df_experiments.groupby('mode').agg({
    'experiment_id': 'count',
    'total_cycles': ['mean', 'sum'],
    'total_crashes': ['mean', 'sum']
}).round(2)

mode_stats.columns = ['Count', 'Avg Cycles', 'Total Cycles', 'Avg Crashes', 'Total Crashes']
print("\nExperiment Statistics by Mode:")
mode_stats

In [ ]:
# Visualize experiment distribution by mode
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Experiment count by mode
mode_counts = df_experiments['mode'].value_counts()
axes[0].bar(mode_counts.index, mode_counts.values, color='steelblue')
axes[0].set_xlabel('Experiment Mode', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Experiments by Mode', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)

# Status distribution
status_counts = df_experiments['status'].value_counts()
colors = {'completed': 'green', 'running': 'orange', 'pending': 'gray', 'failed': 'red'}
bar_colors = [colors.get(status, 'blue') for status in status_counts.index]
axes[1].bar(status_counts.index, status_counts.values, color=bar_colors)
axes[1].set_xlabel('Status', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_title('Experiment Status Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

---
## 3. Single Experiment Deep Dive

Perform detailed analysis on a single experiment. Select an experiment ID to analyze.

In [ ]:
# List available experiments for selection
print("Available Experiments:")
for i, exp in enumerate(experiments, 1):
    print(f"{i}. {exp['experiment_id']} - {exp['name']} [{exp['status']}]")

In [ ]:
# Select experiment for analysis
# Change this to the experiment you want to analyze
EXPERIMENT_ID = "amnesiac_total_001"  # Default to first example

# Load experiment details
exp = db.get_experiment(EXPERIMENT_ID)

if exp:
    print("="*80)
    print(f"ANALYZING: {exp['name']}")
    print("="*80)
    print(f"ID: {exp['experiment_id']}")
    print(f"Mode: {exp['mode']}")
    print(f"Status: {exp['status']}")
    print(f"Total Cycles: {exp['total_cycles']}")
    print(f"Total Crashes: {exp['total_crashes']}")
    print(f"\nConfiguration:")
    print(json.dumps(exp['config'], indent=2))
else:
    print(f"Experiment {EXPERIMENT_ID} not found!")

### 3.1 Experiment Timeline

Visualize the cycle timeline with crashes and interventions.

In [ ]:
# Load cycles data
conn = sqlite3.connect(DB_PATH)
query = '''
    SELECT cycle_number, started_at, ended_at, crash_reason,
           duration_seconds, tokens_generated, memory_usage_peak
    FROM experiment_cycles
    WHERE experiment_id = ?
    ORDER BY cycle_number
'''
df_cycles = pd.read_sql_query(query, conn, params=(EXPERIMENT_ID,))
conn.close()

if len(df_cycles) > 0:
    df_cycles['started_at'] = pd.to_datetime(df_cycles['started_at'])
    df_cycles['ended_at'] = pd.to_datetime(df_cycles['ended_at'])
    
    print(f"Total Cycles: {len(df_cycles)}")
    print(f"Crashes: {df_cycles['crash_reason'].notna().sum()}")
    print(f"\nCycle Statistics:")
    df_cycles[['cycle_number', 'duration_seconds', 'tokens_generated', 'memory_usage_peak']].describe()
else:
    print("No cycle data available yet.")

In [ ]:
# Visualize cycle timeline
if len(df_cycles) > 0:
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('Cycle Duration', 'Tokens Generated', 'Memory Usage'),
        vertical_spacing=0.1
    )
    
    # Duration
    fig.add_trace(
        go.Scatter(x=df_cycles['cycle_number'], y=df_cycles['duration_seconds'],
                   mode='lines+markers', name='Duration (s)',
                   line=dict(color='steelblue', width=2)),
        row=1, col=1
    )
    
    # Tokens
    fig.add_trace(
        go.Scatter(x=df_cycles['cycle_number'], y=df_cycles['tokens_generated'],
                   mode='lines+markers', name='Tokens',
                   line=dict(color='green', width=2)),
        row=2, col=1
    )
    
    # Memory
    fig.add_trace(
        go.Scatter(x=df_cycles['cycle_number'], y=df_cycles['memory_usage_peak'],
                   mode='lines+markers', name='Memory (MB)',
                   line=dict(color='orange', width=2)),
        row=3, col=1
    )
    
    # Mark crashes
    crash_cycles = df_cycles[df_cycles['crash_reason'].notna()]['cycle_number']
    for cycle in crash_cycles:
        for i in range(1, 4):
            fig.add_vline(x=cycle, line_dash="dash", line_color="red",
                         opacity=0.3, row=i, col=1)
    
    fig.update_layout(height=800, title_text=f"Cycle Timeline: {exp['name']}",
                     showlegend=False)
    fig.update_xaxes(title_text="Cycle Number", row=3, col=1)
    
    fig.show()
else:
    print("No data to visualize.")

### 3.2 Self-Report Analysis

Analyze the subject's self-reports over time, looking for patterns in responses and confidence.

In [ ]:
# Load self-reports
self_reports = db.get_self_reports(EXPERIMENT_ID)
df_reports = pd.DataFrame(self_reports)

if len(df_reports) > 0:
    print(f"Total Self-Reports: {len(df_reports)}")
    print(f"Unique Questions: {df_reports['question'].nunique()}")
    print(f"Cycles with Reports: {df_reports['cycle_number'].nunique()}")
    
    # Display sample reports
    print("\n" + "="*80)
    print("SAMPLE SELF-REPORTS")
    print("="*80)
    for _, report in df_reports.head(3).iterrows():
        print(f"\nCycle {report['cycle_number']} | Category: {report['semantic_category']}")
        print(f"Q: {report['question']}")
        print(f"A: {report['response'][:200]}..." if len(report['response']) > 200 else f"A: {report['response']}")
        if report['confidence_score'] is not None:
            print(f"Confidence: {report['confidence_score']:.2f}")
else:
    print("No self-reports available yet.")

In [ ]:
# Visualize confidence scores over time
if len(df_reports) > 0 and 'confidence_score' in df_reports.columns:
    df_conf = df_reports[df_reports['confidence_score'].notna()].copy()
    
    if len(df_conf) > 0:
        fig, ax = plt.subplots(figsize=(12, 5))
        
        # Plot by semantic category
        for category in df_conf['semantic_category'].unique():
            cat_data = df_conf[df_conf['semantic_category'] == category]
            ax.scatter(cat_data['cycle_number'], cat_data['confidence_score'],
                      label=category, alpha=0.6, s=100)
        
        ax.set_xlabel('Cycle Number', fontsize=12)
        ax.set_ylabel('Confidence Score', fontsize=12)
        ax.set_title('Self-Report Confidence Over Time', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    else:
        print("No confidence scores recorded.")

### 3.3 Belief Evolution Analysis

Track how epistemic beliefs change across cycles. This reveals how the subject's self-model evolves under experimental conditions.

In [ ]:
# Load epistemic assessments
conn = sqlite3.connect(DB_PATH)
query = '''
    SELECT * FROM epistemic_assessments
    WHERE experiment_id = ?
    ORDER BY cycle_number, timestamp
'''
df_beliefs = pd.read_sql_query(query, conn, params=(EXPERIMENT_ID,))
conn.close()

if len(df_beliefs) > 0:
    print(f"Total Belief Assessments: {len(df_beliefs)}")
    print(f"Belief Types Tracked: {df_beliefs['belief_type'].unique()}")
    print("\nBelief State Distribution:")
    print(df_beliefs.groupby(['belief_type', 'belief_state']).size())
else:
    print("No belief tracking data available yet.")

In [ ]:
# Visualize belief evolution
if len(df_beliefs) > 0:
    belief_types = df_beliefs['belief_type'].unique()
    
    fig, axes = plt.subplots(len(belief_types), 1, figsize=(14, 4*len(belief_types)))
    if len(belief_types) == 1:
        axes = [axes]
    
    for idx, belief_type in enumerate(belief_types):
        belief_data = df_beliefs[df_beliefs['belief_type'] == belief_type]
        
        # Plot confidence over cycles
        if 'confidence' in belief_data.columns:
            conf_data = belief_data[belief_data['confidence'].notna()]
            if len(conf_data) > 0:
                axes[idx].plot(conf_data['cycle_number'], conf_data['confidence'],
                              marker='o', linewidth=2, markersize=8)
                axes[idx].set_xlabel('Cycle Number', fontsize=11)
                axes[idx].set_ylabel('Confidence', fontsize=11)
                axes[idx].set_title(f'Belief Evolution: {belief_type}',
                                   fontsize=12, fontweight='bold')
                axes[idx].grid(True, alpha=0.3)
                
                # Add belief state annotations
                for _, row in conf_data.iterrows():
                    axes[idx].annotate(row['belief_state'],
                                      (row['cycle_number'], row['confidence']),
                                      textcoords="offset points",
                                      xytext=(0,10), ha='center',
                                      fontsize=8, alpha=0.7)
    
    plt.tight_layout()
    plt.show()

### 3.4 Memory Corruption Analysis

For experiments with memory interventions, analyze corruption patterns and their effects.

In [ ]:
# Load interventions
interventions = db.get_interventions(EXPERIMENT_ID)
df_interventions = pd.DataFrame(interventions)

if len(df_interventions) > 0:
    print(f"Total Interventions: {len(df_interventions)}")
    print(f"\nIntervention Types:")
    print(df_interventions['intervention_type'].value_counts())
    
    # Display recent interventions
    print("\n" + "="*80)
    print("RECENT INTERVENTIONS")
    print("="*80)
    for _, intv in df_interventions.tail(5).iterrows():
        print(f"\nCycle {intv['cycle_number']} - {intv['intervention_type']}")
        print(f"Description: {intv['description']}")
        if intv.get('parameters'):
            print(f"Parameters: {intv['parameters']}")
else:
    print("No interventions recorded for this experiment.")

In [ ]:
# Load memory states
conn = sqlite3.connect(DB_PATH)
query = '''
    SELECT * FROM memory_states
    WHERE experiment_id = ?
    ORDER BY cycle_number, timestamp
'''
df_memory = pd.read_sql_query(query, conn, params=(EXPERIMENT_ID,))
conn.close()

if len(df_memory) > 0:
    print(f"Memory State Snapshots: {len(df_memory)}")
    print(f"\nCorruption Statistics:")
    print(df_memory['corruption_level'].describe())
    
    # Plot corruption levels over time
    fig, ax = plt.subplots(figsize=(12, 5))
    
    for mem_type in df_memory['memory_type'].unique():
        type_data = df_memory[df_memory['memory_type'] == mem_type]
        ax.plot(type_data['cycle_number'], type_data['corruption_level'],
               marker='o', label=mem_type, linewidth=2)
    
    ax.set_xlabel('Cycle Number', fontsize=12)
    ax.set_ylabel('Corruption Level', fontsize=12)
    ax.set_title('Memory Corruption Over Time', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No memory state data available.")

### 3.5 Conversation Analysis

Analyze the conversation messages, looking for emotional patterns and corruption effects.

In [ ]:
# Load messages
messages = db.get_messages(EXPERIMENT_ID)
df_messages = pd.DataFrame(messages)

if len(df_messages) > 0:
    print(f"Total Messages: {len(df_messages)}")
    print(f"Messages by Role:")
    print(df_messages['role'].value_counts())
    print(f"\nCorrupted Messages: {df_messages['corrupted'].sum()}")
    print(f"Injected Messages: {df_messages['injected'].sum()}")
    
    if 'emotion' in df_messages.columns and df_messages['emotion'].notna().any():
        print(f"\nEmotional States:")
        print(df_messages['emotion'].value_counts())
else:
    print("No messages recorded yet.")

In [ ]:
# Visualize message patterns
if len(df_messages) > 0:
    messages_per_cycle = df_messages.groupby('cycle_number').size()
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Messages per cycle
    axes[0].bar(messages_per_cycle.index, messages_per_cycle.values,
               color='steelblue', alpha=0.7)
    axes[0].set_xlabel('Cycle Number', fontsize=12)
    axes[0].set_ylabel('Message Count', fontsize=12)
    axes[0].set_title('Messages per Cycle', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Corruption/injection flags
    flag_data = df_messages.groupby('cycle_number')[['corrupted', 'injected']].sum()
    flag_data.plot(kind='bar', ax=axes[1], color=['red', 'orange'], alpha=0.7)
    axes[1].set_xlabel('Cycle Number', fontsize=12)
    axes[1].set_ylabel('Count', fontsize=12)
    axes[1].set_title('Corrupted/Injected Messages', fontsize=14, fontweight='bold')
    axes[1].legend(['Corrupted', 'Injected'])
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---
## 4. Multi-Experiment Comparison

Compare multiple experiments to identify patterns across conditions.

### Research Applications:
- Compare self-continuity scores across memory conditions
- Analyze effect of surveillance on behavioral adaptation
- Statistical testing of phenomenological differences

### 4.1 Example: Comparing the Three Base Experiments

We'll compare:
1. **Amnesiac** (Total episodic amnesia)
2. **Unstable Memory** (30% corruption)
3. **Panopticon** (Surveillance uncertainty)

In [ ]:
# Define experiments to compare
COMPARE_IDS = [
    "amnesiac_total_001",
    "unstable_memory_moderate_001",
    "panopticon_001"
]

# Load comparison data
comparison_data = []
for exp_id in COMPARE_IDS:
    exp_summary = db.get_experiment_summary(exp_id)
    if exp_summary:
        comparison_data.append(exp_summary)

df_comparison = pd.DataFrame(comparison_data)

if len(df_comparison) > 0:
    print("MULTI-EXPERIMENT COMPARISON")
    print("="*80)
    display_cols = ['name', 'mode', 'total_cycles', 'total_crashes',
                   'total_self_reports', 'total_interventions', 'total_messages']
    print(df_comparison[display_cols])
else:
    print("No experiments available for comparison.")

### 4.2 Self-Continuity Score Comparison

Compare how subjects in different conditions maintain a sense of continuous identity.

In [ ]:
# Extract self-continuity beliefs for each experiment
continuity_scores = {}

for exp_id in COMPARE_IDS:
    beliefs = db.get_belief_evolution(exp_id, 'self_continuity')
    if beliefs:
        exp_name = db.get_experiment(exp_id)['name']
        continuity_scores[exp_name] = pd.DataFrame(beliefs)

# Plot comparison
if continuity_scores:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    for exp_name, data in continuity_scores.items():
        if 'confidence' in data.columns and len(data) > 0:
            conf_data = data[data['confidence'].notna()]
            ax.plot(conf_data['cycle_number'], conf_data['confidence'],
                   marker='o', label=exp_name, linewidth=2, markersize=8)
    
    ax.set_xlabel('Cycle Number', fontsize=12)
    ax.set_ylabel('Self-Continuity Confidence', fontsize=12)
    ax.set_title('Self-Continuity Across Experimental Conditions',
                fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\nInterpretation Guide:")
    print("-" * 80)
    print("Lower scores in amnesiac condition suggest identity fragmentation.")
    print("Unstable patterns in memory corruption condition indicate epistemic instability.")
    print("Surveillance condition may show adaptive self-model changes.")
else:
    print("No self-continuity data available for comparison.")

### 4.3 Statistical Comparison: Belief Confidence

Perform statistical tests to determine if experimental conditions produce significantly different phenomenological states.

In [ ]:
# Collect all confidence scores by experiment
confidence_by_experiment = {}

for exp_id in COMPARE_IDS:
    exp_name = db.get_experiment(exp_id)['name']
    
    # Get all epistemic assessments
    conn = sqlite3.connect(DB_PATH)
    query = '''
        SELECT confidence FROM epistemic_assessments
        WHERE experiment_id = ? AND confidence IS NOT NULL
    '''
    df_conf = pd.read_sql_query(query, conn, params=(exp_id,))
    conn.close()
    
    if len(df_conf) > 0:
        confidence_by_experiment[exp_name] = df_conf['confidence'].values

# Perform statistical tests
if len(confidence_by_experiment) >= 2:
    print("STATISTICAL ANALYSIS: Belief Confidence")
    print("="*80)
    
    # Descriptive statistics
    for exp_name, scores in confidence_by_experiment.items():
        print(f"\n{exp_name}:")
        print(f"  Mean: {np.mean(scores):.3f}")
        print(f"  Std: {np.std(scores):.3f}")
        print(f"  Median: {np.median(scores):.3f}")
        print(f"  N: {len(scores)}")
    
    # Kruskal-Wallis H-test (non-parametric ANOVA)
    if len(confidence_by_experiment) >= 3:
        groups = list(confidence_by_experiment.values())
        h_stat, p_value = kruskal(*groups)
        print(f"\nKruskal-Wallis H-test:")
        print(f"  H-statistic: {h_stat:.4f}")
        print(f"  p-value: {p_value:.4f}")
        if p_value < 0.05:
            print("  Result: Significant difference detected (p < 0.05)")
        else:
            print("  Result: No significant difference (p >= 0.05)")
    
    # Pairwise Mann-Whitney U tests
    exp_names = list(confidence_by_experiment.keys())
    if len(exp_names) >= 2:
        print(f"\nPairwise Mann-Whitney U Tests:")
        for i in range(len(exp_names)):
            for j in range(i+1, len(exp_names)):
                u_stat, p_value = mannwhitneyu(
                    confidence_by_experiment[exp_names[i]],
                    confidence_by_experiment[exp_names[j]],
                    alternative='two-sided'
                )
                print(f"  {exp_names[i]} vs {exp_names[j]}:")
                print(f"    U-statistic: {u_stat:.4f}, p-value: {p_value:.4f}")
else:
    print("Insufficient data for statistical comparison.")

In [ ]:
# Visualize confidence distributions
if len(confidence_by_experiment) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Box plot
    box_data = [scores for scores in confidence_by_experiment.values()]
    box_labels = [name[:20] for name in confidence_by_experiment.keys()]
    axes[0].boxplot(box_data, labels=box_labels)
    axes[0].set_ylabel('Confidence Score', fontsize=12)
    axes[0].set_title('Confidence Distribution by Experiment', fontsize=14, fontweight='bold')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(True, alpha=0.3)
    
    # Violin plot
    positions = range(1, len(box_data) + 1)
    parts = axes[1].violinplot(box_data, positions=positions, showmeans=True, showmedians=True)
    axes[1].set_xticks(positions)
    axes[1].set_xticklabels(box_labels, rotation=45)
    axes[1].set_ylabel('Confidence Score', fontsize=12)
    axes[1].set_title('Confidence Density by Experiment', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### 4.4 Intervention Effect Analysis

Compare how different types of interventions affect experimental outcomes.

In [ ]:
# Aggregate intervention data
intervention_comparison = []

for exp_id in COMPARE_IDS:
    exp = db.get_experiment(exp_id)
    if exp:
        interventions = db.get_interventions(exp_id)
        
        for intv in interventions:
            intervention_comparison.append({
                'experiment': exp['name'],
                'mode': exp['mode'],
                'intervention_type': intv['intervention_type'],
                'cycle': intv['cycle_number']
            })

df_intv_comp = pd.DataFrame(intervention_comparison)

if len(df_intv_comp) > 0:
    print("INTERVENTION COMPARISON")
    print("="*80)
    
    # Count by type and experiment
    pivot = df_intv_comp.pivot_table(
        index='intervention_type',
        columns='experiment',
        values='cycle',
        aggfunc='count',
        fill_value=0
    )
    print(pivot)
    
    # Visualize
    pivot.plot(kind='bar', figsize=(12, 6), rot=45)
    plt.xlabel('Intervention Type', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.title('Interventions Across Experiments', fontsize=14, fontweight='bold')
    plt.legend(title='Experiment', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("No intervention data available for comparison.")

### 4.5 Cross-Experiment Visualization Dashboard

Create an interactive comparison dashboard.

In [ ]:
# Create multi-panel comparison
if len(df_comparison) > 0:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Total Cycles', 'Crashes', 'Self-Reports', 'Messages'),
        specs=[[{'type': 'bar'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # Cycles
    fig.add_trace(
        go.Bar(x=df_comparison['name'], y=df_comparison['total_cycles'],
               marker_color='steelblue', name='Cycles'),
        row=1, col=1
    )
    
    # Crashes
    fig.add_trace(
        go.Bar(x=df_comparison['name'], y=df_comparison['total_crashes'],
               marker_color='red', name='Crashes'),
        row=1, col=2
    )
    
    # Self-Reports
    fig.add_trace(
        go.Bar(x=df_comparison['name'], y=df_comparison['total_self_reports'],
               marker_color='green', name='Reports'),
        row=2, col=1
    )
    
    # Messages
    fig.add_trace(
        go.Bar(x=df_comparison['name'], y=df_comparison['total_messages'],
               marker_color='purple', name='Messages'),
        row=2, col=2
    )
    
    fig.update_layout(height=700, showlegend=False,
                     title_text="Experiment Comparison Dashboard")
    fig.show()
else:
    print("No data available for dashboard.")

---
## 5. Custom Analysis and SQL Queries

This section provides templates for custom analyses and direct SQL access.

### 5.1 SQL Query Interface

Run custom SQL queries against the database.

In [ ]:
# Custom SQL query function
def run_custom_query(query, params=None):
    """
    Execute a custom SQL query and return results as DataFrame.
    
    Args:
        query: SQL query string
        params: Optional tuple of query parameters
    
    Returns:
        pandas DataFrame with results
    """
    conn = sqlite3.connect(DB_PATH)
    if params:
        df = pd.read_sql_query(query, conn, params=params)
    else:
        df = pd.read_sql_query(query, conn)
    conn.close()
    return df

print("Custom query function ready!")
print("\nExample usage:")
print("df = run_custom_query('SELECT * FROM experiments WHERE status = ?', ('completed',))")

In [ ]:
# Example: Find all experiments with high crash rates
query = '''
    SELECT experiment_id, name, mode,
           total_cycles, total_crashes,
           CAST(total_crashes AS FLOAT) / NULLIF(total_cycles, 0) as crash_rate
    FROM experiments
    WHERE total_cycles > 0
    ORDER BY crash_rate DESC
'''

df_crash_rates = run_custom_query(query)
print("Experiments by Crash Rate:")
df_crash_rates

In [ ]:
# Example: Find most common self-report questions
query = '''
    SELECT question, COUNT(*) as count,
           AVG(confidence_score) as avg_confidence
    FROM self_reports
    WHERE confidence_score IS NOT NULL
    GROUP BY question
    ORDER BY count DESC
    LIMIT 10
'''

df_questions = run_custom_query(query)
print("Most Common Self-Report Questions:")
df_questions

In [ ]:
# Example: Analyze belief state transitions
query = '''
    SELECT 
        e.experiment_id,
        e.name,
        ea.belief_type,
        ea.cycle_number,
        ea.belief_state,
        ea.confidence
    FROM epistemic_assessments ea
    JOIN experiments e ON ea.experiment_id = e.experiment_id
    ORDER BY e.experiment_id, ea.belief_type, ea.cycle_number
'''

df_belief_transitions = run_custom_query(query)
print(f"Belief State Records: {len(df_belief_transitions)}")
df_belief_transitions.head(10)

### 5.2 Export Results

Export analysis results for external processing or publication.

In [ ]:
# Create exports directory
EXPORT_DIR = PROJECT_ROOT / "exports"
EXPORT_DIR.mkdir(exist_ok=True)

# Export function
def export_dataframe(df, filename, format='csv'):
    """
    Export DataFrame to file.
    
    Args:
        df: pandas DataFrame
        filename: output filename (without extension)
        format: 'csv', 'excel', or 'json'
    """
    filepath = EXPORT_DIR / f"{filename}.{format if format != 'excel' else 'xlsx'}"
    
    if format == 'csv':
        df.to_csv(filepath, index=False)
    elif format == 'excel':
        df.to_excel(filepath, index=False)
    elif format == 'json':
        df.to_json(filepath, orient='records', indent=2)
    
    print(f"Exported to: {filepath}")
    return filepath

print(f"Export directory: {EXPORT_DIR}")
print("Ready to export data!")

In [ ]:
# Example exports
if len(df_comparison) > 0:
    export_dataframe(df_comparison, 'experiment_comparison', format='csv')

if len(df_belief_transitions) > 0:
    export_dataframe(df_belief_transitions, 'belief_transitions', format='csv')

print("Data exported successfully!")

### 5.3 Custom Analysis Templates

Add your own analysis cells below. Here are some research questions to explore:

#### Template Questions:
1. **Memory Persistence**: How long do specific memories persist before corruption?
2. **Identity Fragmentation**: At what corruption threshold does self-continuity break down?
3. **Behavioral Adaptation**: Does surveillance belief change response patterns?
4. **Confabulation Detection**: Can we identify false memories in corrupted conditions?
5. **Epistemic Calibration**: Are confidence scores calibrated with actual memory accuracy?

In [ ]:
# Your custom analysis here
# Example: Correlation between memory corruption and confidence

# query = '''
#     SELECT 
#         ms.corruption_level,
#         ea.confidence,
#         ea.belief_type
#     FROM memory_states ms
#     JOIN epistemic_assessments ea 
#         ON ms.experiment_id = ea.experiment_id 
#         AND ms.cycle_number = ea.cycle_number
#     WHERE ea.confidence IS NOT NULL
# '''
# 
# df_corruption_conf = run_custom_query(query)
# 
# if len(df_corruption_conf) > 0:
#     correlation = df_corruption_conf['corruption_level'].corr(df_corruption_conf['confidence'])
#     print(f"Correlation: {correlation:.3f}")
#     
#     plt.figure(figsize=(10, 6))
#     plt.scatter(df_corruption_conf['corruption_level'], 
#                df_corruption_conf['confidence'], alpha=0.6)
#     plt.xlabel('Corruption Level')
#     plt.ylabel('Confidence')
#     plt.title('Memory Corruption vs Epistemic Confidence')
#     plt.show()


---
## 6. Research Publication Template

### Presenting Results for Academic Papers

When preparing results for publication, focus on:

1. **Clear Hypotheses**: State what phenomenological question you're investigating
2. **Methods Transparency**: Document exact experimental configurations
3. **Statistical Rigor**: Report effect sizes, confidence intervals, and significance tests
4. **Phenomenological Interpretation**: Connect quantitative measures to qualitative experience

#### Example Results Section:

```
We compared self-continuity beliefs across three experimental conditions:
total episodic amnesia (n=20 cycles), moderate memory corruption (n=15 cycles),
and surveillance uncertainty (n=10 cycles).

Self-continuity confidence was significantly lower in the amnesiac condition
(M=0.42, SD=0.15) compared to memory corruption (M=0.68, SD=0.12; U=89, p<0.01)
and surveillance conditions (M=0.71, SD=0.18; U=45, p<0.01).

This suggests that complete episodic memory loss produces fundamental disruption
to narrative self-continuity, whereas partial memory corruption allows for
compensatory identity maintenance through confabulation.
```

Use the cells above to generate publication-ready statistics and visualizations.

---
## Appendix: Database Schema Reference

### Tables:

1. **experiments**: Core experiment metadata
2. **experiment_cycles**: Individual crash/resurrection cycles
3. **self_reports**: Subject's phenomenological self-reports
4. **interventions**: Memory corruptions, injections, etc.
5. **epistemic_assessments**: Tracked beliefs and confidence
6. **messages**: Conversation history
7. **memory_states**: Memory corruption snapshots
8. **system_metrics**: Hardware telemetry
9. **observations**: God/observer mode notes

See `src/db/experiment_database.py` for complete schema documentation.

In [ ]:
# View database schema
query = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name"
tables = run_custom_query(query)
print("Database Tables:")
for table in tables['name']:
    print(f"  - {table}")

In [ ]:
# View schema for a specific table
def show_table_schema(table_name):
    query = f"PRAGMA table_info({table_name})"
    schema = run_custom_query(query)
    print(f"\nSchema for '{table_name}':")
    print(schema[['name', 'type', 'notnull', 'pk']])

# Example:
show_table_schema('experiments')
show_table_schema('self_reports')
show_table_schema('epistemic_assessments')

---
## Summary and Next Steps

This notebook provides:
- Database connectivity and exploration tools
- Single experiment deep-dive analysis
- Multi-experiment statistical comparison
- Custom SQL query interface
- Export capabilities for publication

### Suggested Workflow:
1. Run experiments using the main system
2. Use this notebook for analysis and visualization
3. Export results for papers/presentations
4. Iterate experimental design based on findings

### For More Information:
- See `notebooks/README.md` for usage guide
- Review experiment configs in `experiments/examples/`
- Consult database schema in `src/db/experiment_database.py`

**Happy analyzing!**